In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D


from tqdm import tqdm 
%matplotlib inline

# plt.style.use("dark_background")

In [4]:
permittivity_of_vacuum = 8.85419e-12
permeability_of_vacuum = 1.25664e-06

In [5]:

c = 1/ np.sqrt( permeability_of_vacuum * permittivity_of_vacuum)
z0 = np.sqrt(permeability_of_vacuum/permittivity_of_vacuum)
f = 1 * 1e8
wave_length = c / f
wave_length

2.9979207053166466

In [6]:
t = 0
dt = 1e-10
nt= 1e4
f = 1 * 1e8
amplitude = 0.1

ndim = 2
Nx = Ny = 100
dx = dy = 0.1 # when wave length is 1.5e6



# set field properties
conductivity = 0 * np.ones((Nx, Ny, 1))
permittivity = permittivity_of_vacuum * np.ones((Nx, Ny, 1))
permeability = permeability_of_vacuum * np.ones((Nx, Ny, 1))

coeff1 = conductivity * dt / (2 * permittivity)
c_ez = (1 - coeff1) / (1 + coeff1) 
c_ezlx = dt/(permittivity * dx * (1 + coeff1))
c_ezly = dt/(permittivity * dy * (1 + coeff1))

c_hxlx = dt/(permeability * dx)
c_hxly = dt/(permeability * dy)

e_field = np.zeros((Nx, Ny, 1)) # only z direction
h_field = np.zeros((Nx, Ny, 2)) # x and y direction

# set excitation properties
ex_x = int(Nx / 2)
ex_y = int(Ny / 2)
t_arr  = np.arange(nt) * dt
ez_result = np.zeros((t_arr.shape[0], Nx, Ny, 1))

c = 1/ np.sqrt( permeability_of_vacuum * permittivity_of_vacuum)
print("check stable", 1/(np.sqrt((1/dx)**2 + (1/dy)**2)) >c * dt)

e_field = np.zeros((Nx, Ny, 1)) # only z direction
h_field = np.zeros((Nx, Ny, 2)) # x and y direction

# set excitation properties
'''
ex_x = int(Nx / 2)
ex_y = int(Ny / 2)
t_arr  = np.arange(nt) * dt
ez_val = amplitude * np.sin(2 * np.pi * f * t_arr)
ez_result = np.zeros((t_arr.shape[0], Nx, Ny, 1))
'''
init_plane_wave = amplitude * np.sin(2 * np.pi * f  / c * dy * np.arange(Ny))
e_field[:,:,0] = init_plane_wave

for k, t in enumerate(t_arr):
    ez_result[k] = e_field
    # calc Ez
    e_field[1:, 1:, 0] = (c_ez[1:, 1:, 0] * e_field[1:, 1:, 0]) + c_ezly[1:, 1:, 0] * (h_field[1:, 1:, 0] - h_field[1:, :Ny-1, 0]) \
        + c_ezlx[1:, 1:, 0] * (h_field[1:, 1:, 1] - h_field[:Nx-1, 1:, 1])    

    # calc Hx
    h_field[:, :Ny-1, 0] = h_field[:, :Ny-1, 0] - c_hxly[:, :Ny-1,0] * (e_field[:, 1:, 0] - e_field[:, :Ny-1, 0])
    # calc Hy
    h_field[:Nx-1, :, 1] -= c_hxlx[:Nx-1, :, 0] * (e_field[1:, :, 0] - e_field[:Nx-1, :, 0])
    #plt.plot(t_arr, ez_val)
    
print(np.sqrt(np.sum(e_field**2)))

check stable True


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: overflow encountered in subtract
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:60: RuntimeWarning: overflow encountered in subtract
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: RuntimeWarning: overflow encountered in add
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in subtract
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in subtract


nan
